In [6]:
import unittest
from unittest.mock import patch
from LogicaGioco import *

1

In [2]:
class TestCharacterFactory(unittest.TestCase):

    def setUp(self):
        # Inizializza i creator prima di ogni test
        self.creator1 = Player1Creator()
        self.creator2 = Player2Creator()

    def test_player1_creation(self):
        player = self.creator1.create_character("Giada", 0)

        # Verifica tipo
        self.assertIsInstance(player, Player1)
        # Verifica attributi
        self.assertEqual(player.nome, "Giada")
        self.assertEqual(player.moralita, 0)

    def test_player2_creation(self):
        player = self.creator2.create_character("Luca", 10)

        self.assertIsInstance(player, Player2)
        self.assertEqual(player.nome, "Luca")
        self.assertEqual(player.moralita, 10)

    def test_players_are_distinct(self):
        # Creo due personaggi con gli stessi dati ma creator diversi
        p1 = self.creator1.create_character("A", 0)
        p2 = self.creator2.create_character("A", 0)

        # Devono essere classi diverse (Player1 vs Player2)
        self.assertNotEqual(type(p1), type(p2))

    def test_multiple_instances(self):
        # Creo due personaggi con lo stesso creator
        p1 = self.creator1.create_character("X", 10)
        p2 = self.creator1.create_character("Y", 0)

        # Devono essere due oggetti distinti in memoria
        self.assertNotEqual(p1, p2)

    @patch("builtins.input", return_value="eroe altruista") 
    def test_altruista_incrementa_correttamente(self, mock_input): 
        player = self.creator1.create_character("Giada", 50) 
        assegna_moralita(player) 
        self.assertEqual(player.moralita, 58) 
        self.assertNotEqual(player.moralita, 50) 

    @patch("builtins.input", return_value="mercenario egoista") 
    def test_egoista_incrementa_correttamente(self, mock_input): 
        player = self.creator1.create_character("Giada", 50) 
        assegna_moralita(player) 
        self.assertEqual(player.moralita, 53) 
        self.assertNotEqual(player.moralita, 50) 
        
    @patch("builtins.input", return_value="anima indifferente") 
    def test_indifferente_incrementa_correttamente(self, mock_input): 
        player = self.creator1.create_character("Giada", 50) 
        assegna_moralita(player) 
        self.assertEqual(player.moralita, 55) 
        self.assertNotEqual(player.moralita, 50)

    @patch("builtins.input", return_value="Giada")
    def test_nome_valido(self, mock_input):
        player = Player1("", 0)
        valida_nome(player, 1)
        self.assertEqual(player.nome, "Giada")

    @patch("builtins.input", return_value="")
    def test_nome_default_player1(self, mock_input):
        player = Player1("", 0)
        valida_nome(player, 1)
        self.assertEqual(player.nome, "Player1")

    @patch("builtins.input", return_value="")
    def test_nome_default_player2(self, mock_input):
        player = Player2("", 0)
        valida_nome(player, 2)
        self.assertEqual(player.nome, "Player2")
        self.assertNotEqual(player.nome, "")
        
if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

..........
----------------------------------------------------------------------
Ran 10 tests in 0.027s

OK


Nome assegnato di default: Player1
Nome assegnato di default: Player2


In [3]:
class TestGameManager(unittest.TestCase):

    def setUp(self):
        """
        Garantisce l'isolamento: Resettiamo l'istanza Singleton prima di ogni test.
        Questo ci permette di testare la creazione 'pulita' ogni volta.
        """
        GameManager._instance = None 

    def test_singleton_is_unique(self):
        """Verifica la proprietà fondaùmentale: una sola istanza (come richiesto dal pattern)."""
        manager1 = GameManager.get_instance() 
        manager2 = GameManager.get_instance() 
        
        # L'Assertion chiave: devono essere lo stesso oggetto in memoria.
        self.assertIs(manager1, manager2, "GameManager non è un Singleton: sono state create due istanze diverse.")

    def test_initial_state_values(self):
        """Verifica che la prima istanza abbia i valori iniziali corretti (da __init__)."""
        manager1 = GameManager.get_instance() 
        
        # Verifica dei vincoli (basati sul tuo codice: Livello 1, 5 Vite)
        self.assertEqual(manager1.livello_corrente, 1)
        self.assertEqual(manager1.vite_rimanenti, 5)
        self.assertEqual(manager1.giocatori, [])

    def test_reset_data_works(self):
        """Verifica che il metodo resetGameData() ripristini i valori iniziali (1 e 5)."""
        manager1 = GameManager.get_instance()
        
        # 1. Modifica lo stato (simulando una partita in corso)
        manager1.livello_corrente = 10 # Cambia il livello
        manager1.vite_rimanenti = 2  # Perde vite
        manager1.giocatori.append("Falso Giocatore")
        
        # 2. Azione: Resetta i dati
        manager1.resetGameData()
        
        # 3. Verifica: Lo stato deve essere tornato ai valori iniziali (5 e 1)
        self.assertEqual(manager1.livello_corrente, 1)
        self.assertEqual(manager1.vite_rimanenti, 5)
        self.assertEqual(manager1.giocatori, [])

if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.............
----------------------------------------------------------------------
Ran 13 tests in 0.031s

OK


Nome assegnato di default: Player1
Nome assegnato di default: Player2
Log: Dati di gioco resettati.
Log: Dati di gioco resettati.
Log: Dati di gioco resettati.
Log: Dati di gioco resettati.


2

In [ ]:
class TestAutoSaveSystem(unittest.TestCase):

    def setUp(self):
        """Prepara un player e un observer puliti prima di ogni test."""
        self.player = Player("Giada", 50)
        
        # CORRETTO: Nessun file path, salviamo in memoria (lista interna)
        self.observer = AutoSaveObserver() 
        self.player.attach(self.observer)

    def test_memento_stores_state(self):
        """Test 15: Verifica che il Memento catturi lo stato esatto."""
        memento = self.player.save_state()
        state = memento.get_state()
        self.assertEqual(state['nome'], "Giada")
        self.assertEqual(state['moralita'], 50)

    def test_restore_state_from_memento(self):
        """Test 16: Verifica che il Player sappia ripristinarsi da un Memento."""
        fake_state = {"nome": "GiadaPassata", "moralita": 10}
        memento = CharacterMemento(fake_state)
        self.player.restore_state(memento)
        
        self.assertEqual(self.player.nome, "GiadaPassata")
        self.assertEqual(self.player.moralita, 10)

    def test_no_save_if_no_change(self):
        """Test 18: Ottimizzazione - Non salvare se il valore è identico."""
        # Facciamo un primo cambio per assicurarci che l'observer funzioni
        self.player.set_moralita(60)
        # Ora l'observer dovrebbe avere 1 memento in memoria.
        # (Assumiamo che l'observer abbia un metodo o attributo per contare la storia, es. len(memento_history))
        # Per ora verifichiamo solo che non crashi o logica simile, 
        # ma idealmente qui controlleremmo: len(self.observer.memento_history)
        
        numero_salvataggi_prima = len(self.observer.memento_history)

        # Azione: Reimposto lo stesso valore (60)
        self.player.set_moralita(60)
        
        # Verifica: Il numero di salvataggi NON deve essere aumentato
        numero_salvataggi_dopo = len(self.observer.memento_history)
        self.assertEqual(numero_salvataggi_prima, numero_salvataggi_dopo)

if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

EEE.............EEEEE
ERROR: test_memento_stores_state (__main__.TestAutoSaveSystem.test_memento_stores_state)
Test 15: Verifica che il Memento catturi lo stato esatto.
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\adria\AppData\Local\Temp\ipykernel_85708\3709871558.py", line 14, in setUp
    self.observer = AutoSaveObserver("test_save.json")
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: AutoSaveObserver.__init__() takes 1 positional argument but 2 were given

ERROR: test_no_save_if_no_change (__main__.TestAutoSaveSystem.test_no_save_if_no_change)
Test 18: Ottimizzazione - Non salvare se il valore è identico.
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\adria\AppData\Local\Temp\ipykernel_85708\3709871558.py", line 14, in setUp
    self.observer = AutoSaveObserver("test_save.json")
                    ^^^^^^^^^^^^^

Nome assegnato di default: Player1
Nome assegnato di default: Player2
Log: Dati di gioco resettati.
Log: Dati di gioco resettati.
Log: Dati di gioco resettati.
Log: Dati di gioco resettati.


3

In [5]:
class TestHealthSystem(unittest.TestCase):

    def setUp(self):
        """
        Prepara l'ambiente prima di ogni test.
        Crea un player standard con 100 di vita massima.
        """
        self.player = Player("TestSubject", 50) 
        # Assumiamo che il costruttore ora inizializzi anche self.health = 100

    def test_initial_health(self):
        """Test 20: Verifica stato iniziale"""
        self.assertEqual(self.player.health, 100)
        self.assertEqual(self.player.max_health, 100)

    def test_take_damage_logic(self):
        """Test 21: Verifica matematica del danno"""
        self.player.take_damage(20)
        self.assertEqual(self.player.health, 80)

    def test_heal_logic(self):
        """Test 23: Verifica matematica della cura"""
        # Prima danneggio per avere spazio di cura
        self.player.take_damage(50) # Vita a 50
        
        self.player.heal(30)
        self.assertEqual(self.player.health, 80)

    def test_health_max_cap(self):
        """Test 25: Limite massimo (Criterio 4)"""
        self.player.take_damage(10) # Vita a 90
        self.player.heal(1000) # Curo tantissimo
        
        # Non deve superare 100
        self.assertEqual(self.player.health, 100)

    def test_health_min_zero(self):
        """Test 26: Limite minimo (Criterio 4)"""
        self.player.take_damage(9999) # Danno letale
        
        # Non deve andare sotto 0
        self.assertEqual(self.player.health, 0)

if __name__ == '__main__':
    unittest.main()

usage: ipykernel_launcher.py [-h] [-v] [-q] [--locals] [--durations N] [-f]
                             [-c] [-b] [-k TESTNAMEPATTERNS]
                             [tests ...]
ipykernel_launcher.py: error: argument -f/--failfast: ignored explicit argument 'c:\\Users\\adria\\AppData\\Roaming\\jupyter\\runtime\\kernel-v301fddd117e3d242177c4754bcde35d2bb98a878c.json'


AttributeError: 'tuple' object has no attribute 'tb_frame'

4

In [ ]:
from inventory import Inventory, InventoryIterator
from player import Player  # Assumiamo che Player sia nel file player.py

class TestInventorySystem(unittest.TestCase):

    def setUp(self):
        """Prepara un inventario pulito prima di ogni test"""
        self.inventory = Inventory()

    def test_inventory_toggle(self):
        """ID 25: Verifica apertura e chiusura"""
        # Parte chiuso
        self.assertFalse(self.inventory.is_open)
        
        # Apro
        self.inventory.toggle()
        self.assertTrue(self.inventory.is_open)
        
        # Chiudo
        self.inventory.toggle()
        self.assertFalse(self.inventory.is_open)

    def test_add_item(self):
        """ID 26: Verifica aggiunta oggetto"""
        self.inventory.add_item("Spada")
        # Verifico che sia nella lista interna (White Box testing)
        self.assertIn("Spada", self.inventory._items)

    def test_iter_connection(self):
        """ID 27: Verifica che Inventory crei un Iteratore"""
        # La funzione iter() chiama dietro le quinte il metodo __iter__()
        iterator = iter(self.inventory)
        self.assertIsInstance(iterator, InventoryIterator)

    def test_iterator_next(self):
        """ID 28: Verifica scorrimento (next)"""
        self.inventory.add_item("A")
        self.inventory.add_item("B")
        
        iterator = iter(self.inventory)
        
        # Primo elemento
        self.assertEqual(next(iterator), "A")
        # Secondo elemento
        self.assertEqual(next(iterator), "B")

    def test_stop_iteration(self):
        """ID 29: Verifica fine lista"""
        self.inventory.add_item("UnicoOggetto")
        iterator = iter(self.inventory)
        
        next(iterator) # Consumo l'oggetto
        
        # Ora deve dare errore StopIteration
        with self.assertRaises(StopIteration):
            next(iterator)

    def test_player_opens_inventory(self):
        """ID 30: Verifica integrazione col Player (Tasto 'i')"""
        # Creo un player (i valori morali/vita non importano qui)
        player = Player("GiadaTest", 50)
        
        # Simulo la pressione del tasto
        player.check_input("i")
        
        # Controllo che l'inventario DEL PLAYER si sia aperto
        self.assertTrue(player.inventory.is_open)

    def test_empty_inventory(self):
        """ID 31: Caso limite - Inventario vuoto"""
        iterator = iter(self.inventory)
        
        # Deve fermarsi subito
        with self.assertRaises(StopIteration):
            next(iterator)

if __name__ == '__main__':
    unittest.main()